In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import pandas as pd

import sys
sys.path.append('..')

from helpers import utils, pipelines, models

from sklearn.model_selection import train_test_split

import multiprocessing

from data_augmentaion.data_augmentator import DataAugmentor 

import json

import time

### Define research parameters

In [6]:
regression_task = False
# continuous_features = []
metric = 'accuracy'
test_size_proportion = 0.2
augment_sample = 0.5

search_pipelines = pipelines.get_classification_pipelines()
search_parameters = models.parameters

settings = [
    {'method': 'random'},
    {'method': 'smote'},
    {'method': 'cf_random'},
    # {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 0.2, 'diversity_weight': 5, 'sparsity_weight': 0.2}},
    # {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 0.2, 'diversity_weight': 5, 'sparsity_weight': 1}},
    {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 5, 'diversity_weight': 0.2, 'sparsity_weight': 0.2}},
    # {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 5, 'diversity_weight': 0.2, 'sparsity_weight': 1}},
    # {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 1, 'diversity_weight': 1, 'sparsity_weight': 0.2}},
    # {'method': 'cf_genetic', 'kw_args': {'proximity_weight': 1, 'diversity_weight': 1, 'sparsity_weight': 1}},
    # {'method': 'cf_kdtree', 'kw_args': {'sparsity_weight': 0.2}},
    # {'method': 'cf_kdtree', 'kw_args': {'sparsity_weight': 1}},
]

In [7]:
df = pd.read_csv(r"..\datasets\cirrhosis.csv")
df.drop(columns="ID",inplace=True)
target = 'Status'

In [8]:
missing_df = df.isna().sum()
dtypes_df = df.dtypes
pd.concat([missing_df, dtypes_df], axis=1)

,0,1
N_Days,0,int64
Status,0,object
Drug,106,object
Age,0,int64
Sex,0,object
Ascites,106,object
Hepatomegaly,106,object
Spiders,106,object
Edema,0,object
Bilirubin,0,float64


In [9]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector


numerical_imputer = SimpleImputer(strategy='mean')#.set_output(transform='pandas')
categorical_imputer = SimpleImputer(strategy='most_frequent')#.set_output(transform='pandas')

final_imputer = ColumnTransformer([
    ('numerical', numerical_imputer, make_column_selector(dtype_include=['int', 'float'])),
    ('categorical', categorical_imputer, make_column_selector(dtype_include=['object'])),
])

df = final_imputer.fit_transform(df)

In [10]:
df = pd.DataFrame(df, columns=[c.split('__')[-1] for c in final_imputer.get_feature_names_out()]).astype(dtypes_df)

In [11]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df[target] = le.fit_transform(df[target])

#### split data

In [12]:
# all data
X = df.drop(target, axis=1)
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_proportion, random_state=42)

In [13]:
continuous_features = X.select_dtypes(include=['number']).columns.tolist()
continuous_features

['N_Days',
 'Age',
 'Bilirubin',
 'Cholesterol',
 'Albumin',
 'Copper',
 'Alk_Phos',
 'SGOT',
 'Tryglicerides',
 'Platelets',
 'Prothrombin',
 'Stage']

In [14]:
display(y_train.value_counts())

0    188
2    125
1     21
Name: Status, dtype: int64

In [15]:
full_results_df = pd.DataFrame()

### Whole dataset scores

In [16]:
print(f'whole adult dataset {metric} scores:')
_, whole_scores = utils.fit_and_evaluate(X_train, y_train, X_test, y_test,
                    search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
whole_dataset_result_df = pd.DataFrame.from_dict(whole_scores, orient='index', columns=[f'full data {metric} score'])
display(whole_dataset_result_df.T)

whole adult dataset accuracy scores:


,lg,rf,xgb
full data accuracy score,0.738095,0.797619,0.761905


In [17]:
full_results_df = pd.concat([full_results_df, whole_dataset_result_df], axis=1)

### little flattening the y-curve

In [18]:
y_train_value_counts = y_train.value_counts()
display(y_train_value_counts)
classes_to_50_gap = 100 - y_train.value_counts()
classes_to_inflate = classes_to_50_gap[classes_to_50_gap>0]
df_inflate = pd.DataFrame()
for c, gap in classes_to_inflate.items():
    df_c_inflate = df[df[target]==c].sample(n=gap, replace=True, random_state=42)
    df_inflate = pd.concat([df_inflate, df_c_inflate])
df_inflate[target].value_counts()


0    188
2    125
1     21
Name: Status, dtype: int64

1    79
Name: Status, dtype: int64

In [19]:
df_full_inflate = pd.concat([df, df_inflate])
X_inflate = df_full_inflate.drop(target, axis=1)
y_inflate = df_full_inflate[target]
X_train_inflate, X_test_inflate, y_train_inflate, y_test_inflate = train_test_split(X_inflate, y_inflate, test_size=test_size_proportion, random_state=42)

In [20]:
print(f'inflate dataset {metric} scores:')
_, inflate_scores = utils.fit_and_evaluate(X_train_inflate, y_train_inflate, X_test_inflate, y_test_inflate,
                    search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
inflate_dataset_result_df = pd.DataFrame.from_dict(inflate_scores, orient='index', columns=[f'inflate data {metric} score'])
display(inflate_dataset_result_df.T)

inflate dataset accuracy scores:


,lg,rf,xgb
inflate data accuracy score,0.7,0.7,0.77


In [21]:
full_results_df = pd.concat([full_results_df, inflate_dataset_result_df], axis=1)

### running experiments

In [22]:
# best_estimators = {}
# best_scores = {}
# for i, s in enumerate(settings):
#     start = time.time()
#     print(f'{i} / {len(settings)}, {s}', end=' ')
#     augmentor = DataAugmentor(X_train, y_train, X_test, y_test,
#                              method=s['method'], regression=regression_task,
#                              continuous_feats=continuous_features,
#                              cf_scoring = metric,
#                              kw_args=s.get('kw_args', {})
#                              )

#     X_train_augmented_balanced, y_train_augmented_balanced = augmentor.augment(balance=True)
#     best_estimators[f'{i}_balanced'], best_scores[f'{i}_balanced'] = \
#     utils.fit_and_evaluate(X_train_augmented_balanced, y_train_augmented_balanced, X_test, y_test,
#                             search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
#     result_df_balanced = pd.DataFrame.from_dict(best_scores[f'{i}_balanced'],
#                                                 orient='index',
#                                                 columns=[f'{json.dumps((list(s.values())))} balanced {metric} score'])

#     X_train_augmented, y_train_augmented = augmentor.augment(balance=False, size=augment_sample)
#     best_estimators[f'{i}'], best_scores[f'{i}'] = \
#         utils.fit_and_evaluate(X_train_augmented, y_train_augmented, X_test, y_test,
#                                search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
#     result_df = pd.DataFrame.from_dict(best_scores[f'{i}'],
#                                        orient='index',
#                                        columns=[f'{json.dumps((list(s.values())))} {metric} score'])
    
#     full_results_df = pd.concat([full_results_df, result_df_balanced, result_df], axis=1)
#     display(full_results_df)
#     print(f'{time.time() - start} seconds for settings {i}')

best_estimators = {}
best_scores = {}
for i, s in enumerate(settings):
    start = time.time()
    print(f'{i+1} / {len(settings)}, {s}', end=' ')
    if 'cf_genetic' in s['method']:
        augmentor = DataAugmentor(X_train_inflate, y_train_inflate, X_test_inflate, y_test_inflate,
                             method=s['method'], regression=regression_task,
                             continuous_feats=continuous_features,
                             cf_scoring = metric,
                             kw_args=s.get('kw_args', {})
                             )

        X_train_augmented_balanced, y_train_augmented_balanced = augmentor.augment(balance=True)
        best_estimators[f'{i}_balanced'], best_scores[f'{i}_balanced'] = \
        utils.fit_and_evaluate(X_train_augmented_balanced, y_train_augmented_balanced, X_test_inflate, y_test_inflate,
                                search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
        result_df_balanced = pd.DataFrame.from_dict(best_scores[f'{i}_balanced'],
                                                    orient='index',
                                                    columns=[f'{json.dumps((list(s.values())))} balanced {metric} score'])

        X_train_augmented, y_train_augmented = augmentor.augment(balance=False, size=augment_sample)
        best_estimators[f'{i}'], best_scores[f'{i}'] = \
            utils.fit_and_evaluate(X_train_augmented, y_train_augmented, X_test, y_test,
                                search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
        result_df = pd.DataFrame.from_dict(best_scores[f'{i}'],
                                        orient='index',
                                        columns=[f'{json.dumps((list(s.values())))} {metric} score'])
        
        full_results_df = pd.concat([full_results_df, result_df_balanced, result_df], axis=1)
        print(f'{time.time() - start} seconds for settings {i}')
    else:
        augmentor = DataAugmentor(X_train, y_train, X_test, y_test,
                                method=s['method'], regression=regression_task,
                                continuous_feats=continuous_features,
                                cf_scoring = metric,
                                kw_args=s.get('kw_args', {})
                                )

        X_train_augmented_balanced, y_train_augmented_balanced = augmentor.augment(balance=True)
        best_estimators[f'{i}_balanced'], best_scores[f'{i}_balanced'] = \
        utils.fit_and_evaluate(X_train_augmented_balanced, y_train_augmented_balanced, X_test, y_test,
                                search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
        result_df_balanced = pd.DataFrame.from_dict(best_scores[f'{i}_balanced'],
                                                    orient='index',
                                                    columns=[f'{json.dumps((list(s.values())))} balanced {metric} score'])

        X_train_augmented, y_train_augmented = augmentor.augment(balance=False, size=augment_sample)
        best_estimators[f'{i}'], best_scores[f'{i}'] = \
            utils.fit_and_evaluate(X_train_augmented, y_train_augmented, X_test, y_test,
                                search_estimators=search_pipelines, search_params=search_parameters, scoring=metric)
        result_df = pd.DataFrame.from_dict(best_scores[f'{i}'],
                                        orient='index',
                                        columns=[f'{json.dumps((list(s.values())))} {metric} score'])
        
        full_results_df = pd.concat([full_results_df, result_df_balanced, result_df], axis=1)
        print(f'{time.time() - start} seconds for settings {i}')



1 / 4, {'method': 'random'} categorical_feats: ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']
3.489854097366333 seconds for settings 0
2 / 4, {'method': 'smote'} categorical_feats: ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']
4.849139213562012 seconds for settings 1
3 / 4, {'method': 'cf_random'} categorical_feats: ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']
0/63


100%|██████████| 1/1 [00:00<00:00,  5.99it/s]


50/63


100%|██████████| 1/1 [00:00<00:00,  9.98it/s]


0/167


...

100/198


100%|██████████| 1/1 [00:00<00:00,  8.19it/s]


150/198


100%|██████████| 1/1 [00:00<00:00,  8.35it/s]


174.6029670238495 seconds for settings 3


In [23]:
full_results_df

,full data accuracy score,inflate data accuracy score,"[""random""] balanced accuracy score","[""random""] accuracy score","[""smote""] balanced accuracy score","[""smote""] accuracy score","[""cf_random""] balanced accuracy score","[""cf_random""] accuracy score","[""cf_genetic"", {""proximity_weight"": 5, ""diversity_weight"": 0.2, ""sparsity_weight"": 0.2, ""total_CFs"": 1, ""desired_class"": 1}] balanced accuracy score","[""cf_genetic"", {""proximity_weight"": 5, ""diversity_weight"": 0.2, ""sparsity_weight"": 0.2, ""total_CFs"": 1, ""desired_class"": 2}] accuracy score"
lg,0.738095,0.70,0.702381,0.761905,0.642857,0.642857,0.702381,0.773810,0.69,0.773810
rf,0.797619,0.70,0.750000,0.809524,0.761905,0.761905,0.690476,0.785714,0.74,0.857143
xgb,0.761905,0.77,0.750000,0.750000,0.738095,0.738095,0.773810,0.773810,0.81,0.892857


In [24]:
full_results_df.to_csv(rf'../log/experiment_multiclass_cirrhosis_{metric}_testsize{test_size_proportion}_augmentsample{augment_sample}.csv')